### Test BIT* Planner on Dual Kuka 14D Env

In [ ]:
%matplotlib inline
import time, sys
sys.path.append('..')
from pb_diff_envs.environment.static.rand_dualkuka14d_env import RandDualKuka14dEnv

import numpy as np
from importlib import reload
import matplotlib.pyplot as plt
import pybullet as p
import pb_diff_envs.objects.static.voxel_group as vv; reload(vv)
from pb_diff_envs.environment.static.dualkuka_rand_vgrp import DualKuka_VoxelRandGroupList

# --- 3 ---
## avg one traj len: , horizon = 
num_groups = 3
num_voxels = 6
hr = 0.22 # 0.2
vr_x = 0.7
vr_y = 0.2
vr_z = (0., 0.)
hExt_range = [hr, hr, hr] # fixed-size obstacle
se_x = 1.0
se_y = 0.5
se_z = (0.3, 0.9) # 0.4 too difficult
seed = 333
## check z
start_end = np.array([[-se_x, se_x], [-se_y, se_y], se_z])
hExt_range = np.array(hExt_range)
void_range = np.array( [(-vr_x, vr_x), (-vr_y, vr_y), vr_z] ) # (0., 0.)

trivial = dict(gen_data=True, samples_per_env=200, gen_num_parts=1,
                dataset_url='./dataset/luotest-dk14d-ipynb.hdf5',
                eval_num_groups=2, eval_num_probs=2, ## dummy placeholder
                )
mazelist_config = dict(planner_timeout=30, min_episode_distance=6*np.pi, interp_density=1.5)
dvg_group = DualKuka_VoxelRandGroupList(RandDualKuka14dEnv, num_groups, num_voxels, start_end, void_range, 
orn_range=None, hExt_range=hExt_range, is_static=True, seed=seed, mazelist_config=mazelist_config, **trivial)

# import gym3
# dvg_group = gym.make('kuka7d-luotest-rand14d-v8', load_mazeEnv=False, gen_data=False) # True

## simple vis of env

In [ ]:
env = dvg_group.create_single_env(2) # 
env.unload_env()
if p.isConnected(): 
    p.disconnect()
env.load(GUI=False) ## if in a headless server: False; local machine: True
p.getContactPoints()
p.stepSimulation()
# env.robot.get_workspace_observation()
plt.imshow(env.render())
plt.show()
# p.configureDebugVisualizer(p.COV_ENABLE_GUI, 0)  # Disable the default GUI controls
# p.configureDebugVisualizer(p.COV_ENABLE_MOUSE_PICKING, 1)  # Enable mouse picking for camera control
p.addUserDebugLine([0, 0, 0], [1, 0, 0], lineColorRGB=[1, 0, 0], lineWidth=6)
p.addUserDebugLine([0, 0, 0], [0, 1, 0], lineColorRGB=[0, 1, 0], lineWidth=6)
p.addUserDebugLine([0, 0, 0], [0, 0, 1], lineColorRGB=[0, 0, 1], lineWidth=6)
no_while = True
while True:
    p.stepSimulation()
    # p.performCollisionDetection()
    c_pts = p.getContactPoints()
    time.sleep(0.1)
    if c_pts is not None and len(c_pts) > 0:
        pass
        # print(p.getContactPoints())
        # time.sleep(2)
        # p.resetJointStatesMultiDof(env.robot_id, np.arange(7), [[0.]]*7)
    if no_while:
        break

In [ ]:
# env.robot.print_joint_states()
# print(env.robot.item_ids)
# print(env.robot_id) # list
# env.robot.limits_high
# # env.robot.limits_low
# print(env._get_joint_pose())
# print(f'{type(env._get_joint_pose())}')


## Sample a problem/episode

In [ ]:
from utils.utils import DotDict
from tqdm import tqdm
env.load(GUI=False)
prev_pose = np.array([0.0] * 14)
env.robot.set_config(prev_pose)
solutions = []
n_traj = 2 # 10
for i in tqdm(range(n_traj)):
    solution, _ = env.sample_1_episode(prev_pose)
    solution = np.array(solution)
    prev_pose = solution[-1]
    solutions.append(solution)

# solutions = np.concatenate(solutions, axis=0)
# result = DotDict(solution=solutions)
# print(solutions.shape)
# print(len(result.solution))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pybullet as p
import pybullet_data
import numpy as np
from utils.utils import save_gif
from IPython.display import HTML
import base64
from objects.trajectory import WaypointLinearTrajectory, WaypointProportionTrajectory
from utils.kuka_utils_luo import SolutionInterp

from importlib import reload
import utils.robogroup_utils_luo  as rul; reload(rul)
from utils.robogroup_utils_luo import robogroup_visualize_traj_luo

noise_config = {'std': 0.000}

sol_interp = SolutionInterp(density=mazelist_config['interp_density'])
sol_i = solutions[0] # which motion trajectory to visualize
result = DotDict(solution=sol_i)
traj_vis = sol_interp(result.solution,)

gifs, ds, vis_dict = robogroup_visualize_traj_luo(env, traj_vis, is_debug=False)


gif_name = f'./test_bit_dualkuka.gif'
save_gif(gifs, gif_name, duration=ds)
b64 = base64.b64encode(open(gif_name, 'rb').read()).decode('ascii')
display(HTML(f'<img src="data:image/gif;base64,{b64}" />'))


In [ ]:
## loop simulation
# p.getBodyInfo(0)
# p.getNumBodies()
# while True:
#     time.sleep(0.05)
#     p.stepSimulation()